In [1]:
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pandahouse
from read_db.CH import Getch

In [2]:
data = Getch(''' SELECT exp_group, 
                        user_id,
                        sum(action = 'like') likes,
                        sum(action = 'view') views,
                        likes/views ctr
                FROM simulator_20220320.feed_actions 
                WHERE toDate(time) between '2022-03-15' and '2022-03-21'
                and exp_group in (0, 3)
                GROUP BY exp_group, user_id
''').df

In [3]:
data.head()

,exp_group,user_id,likes,views,ctr
0,3,115383,9,30,0.300000
1,3,123580,13,48,0.270833
2,0,4944,8,41,0.195122
3,0,4504,5,15,0.333333
4,0,121508,18,88,0.204545


In [5]:
# считаем общий CTR в контрольной группе 
ctr_control = data[data.exp_group == 0].likes.sum() / data[data.exp_group == 0].views.sum()

In [6]:
# посчитаем в обоих группах поюзерную метрику
data['linearized_likes'] = data.likes - ctr_control * data.views

In [7]:
# сделаем t-test на linearized likes 

stats.ttest_ind(data[data.exp_group == 0].linearized_likes,
                data[data.exp_group == 3].linearized_likes,
                equal_var=False)

Ttest_indResult(statistic=-15.21499546090383, pvalue=5.4914249479687664e-52)

In [8]:
# сделаем t-test на поюзерном ctr 

stats.ttest_ind(data[data.exp_group == 0].ctr,
                data[data.exp_group == 3].ctr,
                equal_var=False)

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)

In [9]:
# p-value на linearized likes имеет более высокое значение

In [10]:
# проведем аналогичные действия для 1 и 2 группы 

df = Getch(''' SELECT exp_group, 
                        user_id,
                        sum(action = 'like') likes,
                        sum(action = 'view') views,
                        likes/views ctr
                FROM simulator_20220320.feed_actions 
                WHERE toDate(time) between '2022-03-15' and '2022-03-21'
                and exp_group in (1, 2)
                GROUP BY exp_group, user_id
''').df

In [11]:
df.head()

,exp_group,user_id,likes,views,ctr
0,1,109963,3,15,0.200000
1,1,26117,32,141,0.226950
2,1,138232,18,73,0.246575
3,1,26295,33,122,0.270492
4,1,18392,7,32,0.218750


In [12]:
# считаем общий CTR в контрольной группе 
ctr_control = df[df.exp_group == 1].likes.sum() / df[df.exp_group == 1].views.sum()

In [17]:
# посчитаем в обоих группах поюзерную метрику
df['linearized_likes'] = df.likes - ctr_control * df.views

In [14]:
# сделаем t-test на поюзерном ctr 

stats.ttest_ind(df[df.exp_group == 1].ctr,
                df[df.exp_group == 2].ctr,
                equal_var=False)

Ttest_indResult(statistic=0.7094392041270486, pvalue=0.4780623130874935)

In [18]:
# сделаем t-test на linearized likes 

stats.ttest_ind(df[df.exp_group == 1].linearized_likes,
                df[df.exp_group == 2].linearized_likes,
                equal_var=False)

Ttest_indResult(statistic=6.122579994775973, pvalue=9.439432187037712e-10)

In [19]:
# p-value в случае linearized likes в знаительной мере меньше, чем на поюзерном ctr
# что больше похоже на правду, исходя из анализа в задании два 
# таким образом можно сделать вывод, что linearized likes метрика более чувствительно и это классно